In [7]:
# Manually add the root path before running the test
import sys, os
sys.path.append(os.path.dirname(os.getcwd()))
from utils.helpers import (
    comparator_less,
    prepare_ancilla_cell_validity,
    Indexer
)

from oracle.row_uniqueness import row_uniqueness_circuit

In [9]:
from math import ceil, log2
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile
from qiskit_aer import AerSimulator


# --- 1) PARAMETERS & GRID ---
n = 2
m = 5
# every row has duplicates -> row_constraints_flag should return 1
grid = [
    [6, 2, 5, 1, 3],   
    [4, 1, 5, 6, 3],  
]

# --- 2) SET UP INDEXER + QUBIT POOL ---
# We need:
#  • data: n*n*k
#  • row_flags: n
#  • row_constraints_flag: 1
#  • comparator ancillas: k+1
k = max(1, int(ceil(log2(max(n,m)))))
num_anc = k + 2 * m + n - 1
idx = Indexer(grid, num_anc)

# One flat register for everything
qr = QuantumRegister(idx.total_qubits, name="q")
#cr = ClassicalRegister(1, name="c")   # measure only the final constraint flag
qc = QuantumCircuit(qr, name="row_uniqueness_demo")

idx.initialize_grid(qc)

# --- 4) APPEND THE ROW‐UNIQUENESS ORACLE ---
# Flips row_flags[i] if row i has duplicates, then collapses into idx.row_flag()
row_uniqueness_circuit(qc, qr, idx)

# --- 5) MEASURE JUST THE GLOBAL ROW‐CONSTRAINT FLAG ---
#qc.measure(qr[idx.row_flag()], cr[0])
qc.measure_all()

# --- 6) SIMULATE & PRINT RESULTS ---
sim = AerSimulator(method="matrix_product_state")
tcirc = transpile(qc, sim, optimization_level=3)
result = sim.run(tcirc, shots=1).result()
counts = result.get_counts()

print("Total qubits:", qc.num_qubits)
print("Counts for row_constraints_flag:", counts)


Total qubits: 48
Counts for row_constraints_flag: {'000000000000000000011110101001100011001101010110': 1}


In [ ]:
# --- after you’ve run and obtained `result.get_counts()` ---
counts    = result.get_counts()
bitstring = next(iter(counts))            # e.g. '01000101…'
# Qiskit’s bitstrings are MSB→LSB (left→right), qubit 0 is the rightmost bit
bits      = list(map(int, bitstring[::-1]))  # now bits[i] is the value of qubit i

n, m, k = idx.n, idx.m, idx.k

# 1) print out each qubit’s role and its measured value
print("Measured qubits:")
for q in range(idx.total_qubits):
    role = idx.pretty(q)
    print(f" q[{q:2d}] = {bits[q]}   ⟶ {role}")

# 2) reconstruct the grid from the data‐region into a 2D list
recovered = [[None for _ in range(m)] for _ in range(n)]
for i in range(n):
    for j in range(m):
        v = 0
        for b in range(k):
            bitval = bits[idx.data(i, j, b)]
            v |= (bitval << b)
        recovered[i][j] = v

# 3) print the grid row by row
print("\nReconstructed grid values:")
for i, row in enumerate(recovered):
    print(f" row {i:2d}: {row}")

# 4) check the final row‐constraint flag
rf = idx.row_flag()
print(f"\nrow_constraints_flag (q[{rf}]) = {bits[rf]}")


Measured qubits:
 q[ 0] = 0   ⟶ data(0,0,0)
 q[ 1] = 1   ⟶ data(0,0,1)
 q[ 2] = 1   ⟶ data(0,0,2)
 q[ 3] = 0   ⟶ data(0,1,0)
 q[ 4] = 1   ⟶ data(0,1,1)
 q[ 5] = 0   ⟶ data(0,1,2)
 q[ 6] = 1   ⟶ data(0,2,0)
 q[ 7] = 0   ⟶ data(0,2,1)
 q[ 8] = 1   ⟶ data(0,2,2)
 q[ 9] = 1   ⟶ data(0,3,0)
 q[10] = 0   ⟶ data(0,3,1)
 q[11] = 0   ⟶ data(0,3,2)
 q[12] = 1   ⟶ data(0,4,0)
 q[13] = 1   ⟶ data(0,4,1)
 q[14] = 0   ⟶ data(0,4,2)
 q[15] = 0   ⟶ data(1,0,0)
 q[16] = 0   ⟶ data(1,0,1)
 q[17] = 1   ⟶ data(1,0,2)
 q[18] = 1   ⟶ data(1,1,0)
 q[19] = 0   ⟶ data(1,1,1)
 q[20] = 0   ⟶ data(1,1,2)
 q[21] = 1   ⟶ data(1,2,0)
 q[22] = 0   ⟶ data(1,2,1)
 q[23] = 1   ⟶ data(1,2,2)
 q[24] = 0   ⟶ data(1,3,0)
 q[25] = 1   ⟶ data(1,3,1)
 q[26] = 1   ⟶ data(1,3,2)
 q[27] = 1   ⟶ data(1,4,0)
 q[28] = 1   ⟶ data(1,4,1)
 q[29] = 0   ⟶ data(1,4,2)
 q[30] = 0   ⟶ row_flag
 q[31] = 0   ⟶ col_flag
 q[32] = 0   ⟶ cell_valid_flag
 q[33] = 0   ⟶ global_flag
 q[34] = 0   ⟶ ancilla(0)
 q[35] = 0   ⟶ ancilla(1)
 q[36] = 0   ⟶ 